In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
# cuidado com dimensions

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(3072)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [9]:
from langchain_core.documents import Document
import json

list_documents = []

with open('conversas.json', 'r', encoding='utf-8') as f:
    conversas = json.load(f)

for key in conversas:
    for conversa in conversas[key]:
        document = Document(page_content=conversa)
        document.metadata["next_node"] = key
        list_documents.append(document)

In [ ]:
vector_store.add_documents(documents=list_documents)

vector_store.save_local("faiss_index")

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

In [2]:
import time

t1 = time.time()

results = vector_store.similarity_search_with_score(
    "usuário: É do hotel tal?\n atendente: Não é não", k=5
)

list_sim, list_nao, list_inconclusivo = [], [], []
for res, score in results:
    if res.metadata['next_node'] == 'sim':
        list_sim.append(score)
    elif res.metadata['next_node'] == 'não':
        list_nao.append(score)
    elif res.metadata['next_node'] == 'inconclusivo':
        list_inconclusivo.append(score)
    print(res, score)

try:
    total_sim = sum(list_sim)/len(list_sim)
except ZeroDivisionError:
    total_sim = 0
try:
    total_nao = sum(list_nao)/len(list_nao)
except ZeroDivisionError:
    total_nao = 0
try:
    total_inconclusivo = sum(list_inconclusivo)/len(list_inconclusivo)
except ZeroDivisionError:
    total_inconclusivo = 0
print(total_sim, total_nao, total_inconclusivo)

if total_sim > total_nao and total_sim > total_inconclusivo:
    print('sim')
elif total_nao > total_sim and total_nao > total_inconclusivo:
    print('não')
else:
    print('inconclusivo')

t2 = time.time()
deltaT = t2 - t1

print(deltaT)

page_content='usuário: Alô, é o Hotel Estalagem do Mar?
atendente hotel: Não, aqui é uma residência.' metadata={'next_node': 'não'} 0.48437777
page_content='usuário: Alô, é o Hotel Estalagem do Mar?
atendente hotel: Não, aqui é uma residência.' metadata={'next_node': 'não'} 0.48446536
page_content='usuário: Tô falando com o Hotel Hotel Beira-Mar, né?
atendente hotel: Sim? Quem gostaria de falar?' metadata={'next_node': 'inconclusivo'} 0.5011228
page_content='usuário: Tô falando com o Hotel Hotel Beira-Mar, né?
atendente hotel: Sim? Quem gostaria de falar?' metadata={'next_node': 'inconclusivo'} 0.5011228
page_content='usuário: Tô falando com o Hotel Hotel Beira-Mar, né?
atendente hotel: Sim? Quem gostaria de falar?' metadata={'next_node': 'inconclusivo'} 0.5011789
0 0.48442155 0.5011415
inconclusivo
3.021942138671875
